In [149]:
import json
import pandas as pd 
import sys
from collections import defaultdict
import numpy as np

sys.path.append('..')
import ENVIRONMENT_VARIABLES as EV

In [150]:
years_analysed = [2024]
teams = ["Broncos", "Roosters", "Wests Tigers", "Rabbitohs", "Storm", "Eels", "Raiders", "Knights", "Dragons", "Sea Eagles", "Panthers", "Sharks", "Bulldogs", "Dolphins", "Titans", "Cowboys", "Warriors"]
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round"]
years =  [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2022, 2023]

# Match Data

In [151]:
teams = EV.TEAMS
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round", "Halftime_Attack", "Halftime_Defense"]
years =  [2024]
TOTAL_ROUNDS = 8

# Create a DataFrame with columns representing combinations of team and variable names
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in variables])

In [152]:

# Initialize an empty dictionary to store data for each year
years_arr = {}

# Open the JSON file containing NRL data
with open('./data/nrl_data_2024.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    
    # Extract NRL data from the loaded JSON
    data = data['NRL']
    
    # Iterate over each year in the years list
    for year in years:
        # Extract data for the current year and store it in the years_arr dictionary
        # Note: years.index(year) returns the index of the current year in the years list
        #       This index is then used to access the corresponding data for that year
        years_arr[year] = data[years.index(year)][str(year)]


In [153]:
round_game_data = defaultdict(dict)

with open(f'./data/nrl_detailed_match_data_{years_analysed[0]}.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    data = data['NRL']
    for round in range(0, TOTAL_ROUNDS):
        round_data = data[round]
        for key, value in round_data.items():
            game_datas = round_data[key]
            for game in game_datas:
                # Now the game 
                game_name = list(game.keys())[0]
                round_game_data[round][game_name] = game[game_name]

In [154]:
round_game_data[7]

{'Warriors v Titans': {'match': {'overall_first_try_scorer': 'Charnze Nicoll-Klokstad',
   'overall_first_try_minute': "3'",
   'overall_first_try_round': 'warriors',
   'ref_names': ['Wyatt Raymond', 'Matt Noyen', 'Liam Kennedy'],
   'ref_positions': ['Referee', 'Touch Judge', 'Senior Review Official'],
   'main_ref': 'Wyatt Raymond',
   'ground_condition': 'Good',
   'weather_condition': ''},
  'home': {'possession': '52%',
   'try_names': ['Charnze Nicoll-Klokstad',
    'Addin Fonua-Blake',
    'Addin Fonua-Blake',
    'Rocco Berry'],
   'try_minutes': ["3'", "6'", "57'", "62'"],
   'first_try_scorer': 'Charnze Nicoll-Klokstad',
   'first_try_time': "3'",
   'time_in_possession': '29:38',
   'all_runs': '197',
   'all_run_metres': '1,930',
   'post_contact_metres': '764',
   'line_breaks': '5',
   'tackle_breaks': '35',
   'average_set_distance': '49.5',
   'kick_return_metres': '267',
   'offloads': '7',
   'receipts': '455',
   'total_passes': '246',
   'dummy_passes': '12',
   'k

In [155]:

# Initialize an empty list to store data for all rounds
all_store = []

# Iterate over each year in the years list
for year in years:
    # Iterate over each round (assuming 26 rounds)
    for round in range(0, TOTAL_ROUNDS):
        try:
            # Extract data for the current round
            round_data = years_arr[year][round][str(round+1)]
            
            # Create an empty feature array 
            round_store = np.zeros([len(teams)*len(variables)], dtype=int)
            round_teams = []
            
            # Iterate over each game in the round data
            for game in round_data:
                # Extract information about the game
                h_team = game['Home']
                h_score = int(game['Home_Score'])
                a_team = game['Away']
                a_score = int(game['Away_Score'])
                
                h_team_name = h_team.replace(" ", "-")
                a_team_name = a_team.replace(" ", "-")
                

                rgd = round_game_data[round][f"{h_team} v {a_team}"]
                match_data_away = rgd['away']['half_time']
                match_data_home = rgd['home']['half_time']
                

                print(f"{round} {h_team_name} {a_team_name} {match_data_away}")
                


                
                # home_team_data = rgd['home']
                # home_penalitites = home_team_data['penalties_conceded']
                
                
                # away_team_data = rgd['away']
                # away_penalitites = away_team_data['penalties_conceded']
                
                
                # Determine win or lose for each team
                h_team_win, a_team_win = h_score >= a_score, a_score >= h_score
                
                # Determine home team status
                h_home, a_home = 1, 0
                
                # Determine versus index
                h_versus, a_versus= teams.index(a_team), teams.index(h_team)
                
                # Determine defense (points let in)
                h_team_defense = a_score
                a_team_defense = h_score  
                
                # Determine attack points scored
                h_team_attack = h_score 
                a_team_attack = a_score   
                
                # Determine margin
                h_team_margin =  h_score - a_score   
                a_team_margin =  a_score - h_score        
                
                # Keep track of which teams played to work out which teams had a bye 
                round_teams.append(h_team)
                round_teams.append(a_team)
                
                # Find the index of the team in the overarching array 
                a_team_idx = teams.index(a_team)
                h_team_idx = teams.index(h_team)
                
                # Determine feature map index
                a_team_idx_fm = a_team_idx * len(variables)
                h_team_idx_fm = h_team_idx * len(variables)
                
                # Populate data for away team
                for idx, data in zip(range(a_team_idx_fm, a_team_idx_fm + 10), 
                                    [year, a_team_win, a_team_defense, a_team_attack, a_team_margin, a_home, a_versus, round + 1, match_data_away, match_data_home]):
                    round_store[idx] = data

                # Populate data for home team
                for idx, data in zip(range(h_team_idx_fm, h_team_idx_fm + 10),
                                    [year, h_team_win, h_team_defense, h_team_attack, h_team_margin, h_home, h_versus, round + 1, match_data_home, match_data_away]):
                    round_store[idx] = data
                
            # Determine teams with a bye
            bye_teams = list(set(teams) - set(round_teams))
            
            # Assign values for teams with a bye
            for bye_team in bye_teams:
                b_team_idx = teams.index(bye_team)
                b_team_idx_fm = b_team_idx * len(variables)
                round_store[b_team_idx_fm] = year
                round_store[b_team_idx_fm+1] = -1
                round_store[b_team_idx_fm+2] = -1
                round_store[b_team_idx_fm+3] = -1
                round_store[b_team_idx_fm+4] = 0
                round_store[b_team_idx_fm+5] = -1
                round_store[b_team_idx_fm+6] = -1
                round_store[b_team_idx_fm+7] = round+1
                round_store[b_team_idx_fm+8] = -1
                
                
            # Append the round data to the all_store list
            all_store.append(round_store)
            
            # Add the new row to the DataFrame using loc
            df.loc[len(df)] = round_store
        except Exception as ex:
            print(ex)
        

0 Sea-Eagles Rabbitohs 10
0 Roosters Broncos 4
0 Knights Raiders 8
0 Warriors Sharks 6
0 Storm Panthers 0
0 Eels Bulldogs 0
0 Titans Dragons 6
0 Dolphins Cowboys 31
1 Broncos Rabbitohs 0
1 Sharks Bulldogs 6
1 Panthers Eels 18
1 Raiders Wests-Tigers 12
1 Cowboys Knights 12
1 Storm Warriors 6
1 Sea-Eagles Roosters 10
1 Dolphins Dragons 0
2 Panthers Broncos 2
2 Warriors Raiders 4
2 Roosters Rabbitohs 0
2 Bulldogs Titans 0
2 Dragons Cowboys 22
2 Wests-Tigers Sharks 6
2 Eels Sea-Eagles 18
2 Knights Storm 0
3 Roosters Panthers 14
3 Rabbitohs Bulldogs 12
3 Broncos Cowboys 12
3 Dragons Sea-Eagles 6
3 Titans Dolphins 12
3 Warriors Knights 6
3 Sharks Raiders 18
3 Eels Wests-Tigers 6
4 Storm Broncos 18
4 Bulldogs Roosters 1
4 Knights Dragons 10
4 Rabbitohs Warriors 18
4 Sea-Eagles Panthers 6
4 Dolphins Wests-Tigers 0
4 Cowboys Titans 0
4 Raiders Eels 0
5 Knights Roosters 16
5 Storm Bulldogs 0
5 Broncos Dolphins 4
5 Warriors Sea-Eagles 16
5 Eels Cowboys 10
5 Rabbitohs Sharks 16
5 Wests-Tigers Drag

In [156]:
filtered_columns = [col for col in df.columns if 'halftime_attack' in col.lower() or 'halftime_defense' in col.lower()or 'attack' in col.lower() or 'defense' in col.lower()]
filtered_df = df[filtered_columns]

In [157]:
for team in EV.TEAMS:
    # Calculate the difference between Attack and Halftime columns
    attack_col = f"{team} Attack"
    attack_def_col = f"{team} Defense"
    halftime_col = f"{team} Halftime_Attack"
    halftime_def_col = f"{team} Halftime_Defense"
    second_half_col = f"{team} SecondHalf_Attack"
    second_half_def_col = f"{team} SecondHalf_Attack"
    filtered_df[second_half_col] = filtered_df[attack_col] - filtered_df[halftime_col]
    filtered_df[second_half_def_col] = filtered_df[attack_def_col] - filtered_df[halftime_def_col]

filtered_df = filtered_df.reindex(sorted(filtered_df.columns), axis=1)


C:\Users\beauh\AppData\Local\Temp\ipykernel_26060\603877289.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[second_half_col] = filtered_df[attack_col] - filtered_df[halftime_col]
C:\Users\beauh\AppData\Local\Temp\ipykernel_26060\603877289.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[second_half_def_col] = filtered_df[attack_def_col] - filtered_df[halftime_def_col]
C:\Users\beauh\AppData\Local\Temp\ipykernel_26060\603877289.py:9: SettingWithCopyWarning: 
A value is trying 

# Player Data

In [158]:
player_variables =["Name"] +EV.PLAYER_LABELS

In [159]:
data

'18'

In [160]:
# Initialize an empty dictionary to store player statistics for each year
years_arr = {}

# Open the JSON file containing player statistics data for a specific year
with open('./data/player_statistics_2024.json', 'r') as file:
    data = json.load(file)  # Load JSON data from the file
    data = data['PlayerStats']  # Extract the 'PlayerStats' key from the JSON data
    for year in years_analysed:  # Iterate over the years to be analyzed
        # Store player statistics data for the current year in the 'years_arr' dictionary
        years_arr[year] = data[years_analysed.index(year)][str(year)]

In [161]:
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in player_variables])

In [162]:
len(years_arr[2024][0])

1

In [163]:

player_stats = defaultdict(list)  # Dictionary to store player statistics

for year in years_analysed:  # Loop through each year to analyze
    for i in range(0, TOTAL_ROUNDS):  # Loop through each round (assuming there are 26 rounds)
        try:
            round = years_arr[year][i]  # Get data for a particular round
            round = round[str(i)]  # Assuming round is stored as a string key
            for round_game in round:  # Loop through each game in the round
                for game in round_game:  # Loop through each team's performance in the game
                    # Splitting game data to extract year, round, home team, and away team
                    game_split = game.split("-")
                    game_year, game_round = game_split[0], game_split[1]

                    game_split = game.split("v")
                    home_team = " ".join(game_split[0].split("-")[2:]).replace("-", " ").strip()
                    away_team = " ".join(game_split[-1:]).replace("-", " ").strip()
                    
                    print(game)

                    players = round_game[game]  # List of players in the game
                    player_round_stats = {}  # Dictionary to store player statistics for the game
                    for player in players:  # Loop through each player's stats
                        vals = [player[val] for val in player_variables]  # Extract required stats for each player
                        player_round_stats[vals[0]] = vals[1:]  # Store player stats with player ID as key
                    
                    player_round_stats = list(player_round_stats.items())  # Convert player stats to list of tuples
                    # Split player stats into home and away team stats
                    player_round_stats_home, player_round_stats_away = player_round_stats[:18], player_round_stats[18:]
                    
                    # Append home team player stats to player_stats dictionary
                    for player in player_round_stats_home:
                        player_stats[player[0]].append([player[1], game_round, home_team, away_team])
                    # Append away team player stats to player_stats dictionary
                    for player in player_round_stats_away:
                        player_stats[player[0]].append([player[1], game_round, away_team, home_team])
        except IndexError as ex:
            print(f"Run out of round data, round: {i}, \n{ex}")
                    
        except Exception as ex:
            print(f"{ex}\tround: {i}:{game}")  # Print any exceptions that occur during processing


2024-1-Sea-Eagles-v-Rabbitohs
2024-1-Roosters-v-Broncos
2024-1-Knights-v-Raiders
2024-1-Warriors-v-Sharks
2024-1-Storm-v-Panthers
2024-1-Eels-v-Bulldogs
2024-1-Titans-v-Dragons
2024-1-Dolphins-v-Cowboys
2024-2-Broncos-v-Rabbitohs
2024-2-Sharks-v-Bulldogs
2024-2-Panthers-v-Eels
2024-2-Raiders-v-Wests-Tigers
2024-2-Cowboys-v-Knights
2024-2-Storm-v-Warriors
2024-2-Sea-Eagles-v-Roosters
2024-2-Dolphins-v-Dragons
2024-3-Panthers-v-Broncos
2024-3-Warriors-v-Raiders
2024-3-Roosters-v-Rabbitohs
2024-3-Bulldogs-v-Titans
2024-3-Dragons-v-Cowboys
2024-3-Wests-Tigers-v-Sharks
2024-3-Eels-v-Sea-Eagles
2024-3-Knights-v-Storm
2024-4-Roosters-v-Panthers
2024-4-Rabbitohs-v-Bulldogs
2024-4-Broncos-v-Cowboys
2024-4-Dragons-v-Sea-Eagles
2024-4-Titans-v-Dolphins
2024-4-Warriors-v-Knights
2024-4-Sharks-v-Raiders
2024-4-Eels-v-Wests-Tigers
2024-5-Storm-v-Broncos
2024-5-Bulldogs-v-Roosters
2024-5-Knights-v-Dragons
2024-5-Rabbitohs-v-Warriors
2024-5-Sea-Eagles-v-Panthers
2024-5-Dolphins-v-Wests-Tigers
2024-5-C

In [164]:
# Combine player labels with additional information
headers = EV.PLAYER_LABELS + ["Round", "Team", "Opposition"]
players = {}

# Loop through each player and their statistics
for player, values in player_stats.items():
    print(player)
    # Combine round statistics with additional information
    player_stat = [[*round[0], *round[1:]] for round in values]
    # Create a DataFrame from the combined statistics and headers
    df = pd.DataFrame(player_stat, columns=headers)
    players[player] = df

Tom Trbojevic
Jason Saab
Tolutau Koula
Reuben Garrick
Jaxson Paulo
Luke Brooks
Daly Cherry-Evans
Taniela Paseka
Lachlan Croker
Josh Aloiai
Haumole Olakau'atu
Ben Trbojevic
Jake Trbojevic
Karl Lawton
Corey Waddell
Ethan Bullemor
Nathan Brown
Jake Arthur
Latrell Mitchell
Alex Johnston
Isaiah Tass
Richard Kennar
Jacob Gagai
Cody Walker
Lachlan Ilias
Tevita Tatola
Damien Cook
Sean Keppie
Keaon Koloamatangi
Jai Arrow
Cameron Murray
Siliva Havili
Jacob Host
Davvy Moale
Thomas Burgess
Dean Hawkins
James Tedesco
Daniel Tupou
Joseph Manu
Joseph-Aukuso Suaalii
Luke Keary
Sam Walker
Spencer Leniu
Brandon Smith
Lindsay Collins
Siua Wong
Sitili Tupouniua
Victor Radley
Sandon Smith
Naufahu Whyte
Nat Butcher
Terrell May
Fetalaiga Pauga
Connor Watson
Reece Walsh
Jesse Arthars
Kotoni Staggs
Selwyn Cobbo
Deine Mariner
Ezra Mam
Adam Reynolds
Corey Jensen
Billy Walters
Payne Haas
Brendan Piakura
Jordan Riki
Patrick Carrigan
Tyson Smoothy
Fletcher Baker
Kobe Hetherington
Corey Oates
Martin Taupau
Kalyn Pon

In [165]:
players['Clinton Gutherson']

,Number,Position,Mins Played,Points,Tries,Conversions,Conversion Attempts,Penalty Goals,Goal Conversion Rate,1 Point Field Goals,...,Ruck Infringements,Inside 10 Metres,On Report,Sin Bins,Send Offs,Stint One,Stint Two,Round,Team,Opposition
0,1,Fullback,80:00,8,-,3,3,1,100%,-,...,-,-,-,-,-,80:00,-,1,Eels,Bulldogs
1,1,Fullback,80:00,4,-,2,2,-,100%,-,...,-,-,-,-,-,80:00,-,2,Eels,Panthers
2,1,Fullback,80:00,6,-,3,5,-,60%,-,...,-,-,-,-,-,80:00,-,3,Eels,Sea Eagles
3,1,Fullback,80:00,12,1,1,1,3,80%,-,...,-,-,-,-,-,80:00,-,4,Eels,Wests Tigers
4,1,Fullback,80:00,-,-,-,2,-,-,-,...,-,-,-,-,-,80:00,-,5,Eels,Raiders
5,1,Fullback,80:00,10,-,4,4,1,83.33%,-,...,-,-,-,-,-,80:00,-,6,Eels,Cowboys
6,1,Fullback,80:00,4,-,1,1,1,100%,-,...,-,-,-,-,-,80:00,-,7,Eels,Dolphins
7,1,Fullback,80:00,2,-,-,3,1,25%,-,...,-,-,-,-,-,80:00,-,8,Eels,Sea Eagles


In [166]:

# team_1_name = "Dragons"
# team_1 =['Tyrell Sloan', 'Zac Lomax', 'Mosese Suli', 'Jack Bird', 'Mikaele Ravalawa', 'Kyle Flanagan',
#        'Ben Hunt', 'Jack De Belin', 'Jesse Marschke',
#        'Blake Lawrie', 'Luciano Leilua', 'Raymond Faitala-Mariner', 'Tom Eisenhuth']

# team_2_name = "Cowboys"
# team_2 =['Scott Drinkwater', 'Kyle Feldt', 'Valentine Holmes', 'Zac Laybutt', 'Murray Taulagi', 
#          'Tom Dearden', 'Chad Townsend', 'Jordan McLean', 'Reece Robson', 'Jason Taumalolo',
#          'Kulikefu Finefeuiaki', 'Jeremiah Nanai', 'Reuben Cotter']

# team_1_name = "Bulldogs"
# team_1 =['Blake Taaffe', 'Blake Wilson','Jacob Kiraz', 'Stephen Crichton', 'Josh Addo-Carr', 'Matt Burton',
#          'Drew Hutchison', 'Max King', 'Reed Mahoney', 'Liam Knight', 'Villame Kikau',
#          'Jacob Preston', 'Jaeman Salmon']

# team_2_name = "Titans"
# team_2 =['Keano Kini', 'Alofiana Khan-Pereira', 'Brian Kelly', 'AJ Brimson', 'Phillip Sami',
#          'Kieran Foran', 'Tanah Boyd', 'Moeaki Fotuaika', 'Chris Randall', 'Tino Fa\'asuamaleaui', 'Klese Haas',
#          'Beau Fermor', 'Jaimin Jolliffe']


# team_1_name = "Warriors"
# team_1 =['Taine Tuaupiki', 'Dallin Watene-Zelezniak', 'Rocco Berry', 'Roger Tuivasa-Sheck', 'Marcelo Montoya',
#          'Luke Metcalf', 'Shaun Johnson', 'Addin Fonua-Blake', 'Freddy Lussick', 'Mitchell Barnett',
#          'Jackson Ford', 'Kurt Capewell', 'Tohu Harris']

# team_2_name = "Raiders"
# team_2 =['Jordan Rapana' ,'Nick Cotric', 'Matthew Timoko', 'Albert Hopoate', 'Xavier Savage', 'Ethan Strange',
#          'Jamal Fogarty', 'Josh Papali\'i', 'Danny Levi', 'Joseph Tapine', 'Hudson Young', 'Elliott Whitehead',
#          'Morgan Smithies']

# team_1_name = "Wests Tigers"
# team_1 = ['Jahream Bula', 'Charlie Staines', 'Solomona Faataape', 'Justin Olam', 'Junior Tupou', 
#           'Lachlan Galvin', 'Aidan Sezer', 'Stefano Utoikamanu', 'Apisai Koroisau', 'David Klemmer', 
#           'Isaiah Papali\'i', 'John Bateman', 'Fonua Pole']
# team_2_name = "Sharks"
# team_2 = ['William Kennedy', 'Sione Katoa', 'Jesse Ramien' ,'Siosifa Talakai', 'Ronaldo Mulitalo', 'Braydon Trindall',
#           'Nicholas Hynes', 'Toby Rudolf', 'Blayke Brailey', 'Oregon Kaufusi', 'Jack Williams',
#           'Teig Wilton', 'Cameron McInnes']


team_1_name = "Eels"
team_1 = ['Clinton Gutherson', 'Morgan Harper', 'Will Penisini', 'Blaize Talagi', 'Sean Russell',
          'Dylan Brown', 'Mitchell Moses', 'Reagan Campbell-Gillard', 'Joey Lussick', 'Junior Paulo',
          'Shaun Lane', 'Bryce Cartwright', 'J\'maine Hopgood']

team_2_name  = "Sea Eagles"
team_2 = ['Tom Trbojevic', 'Raymond Tuaimalo Vaega', 'Tolutau Koula', 'Reuben Garrick',
          'Jaxson Paulo', 'Luke Brooks', 'Daly Cherry-Evans', 'Taniela Paseka', 'Lachlan Croker', 'Josh Aloiai',
          'Haumole Olakau\'atu', 'Ben Trbojevic', 'Jake Trobojevic']



names = {1: 'Fullback', 2: 'Winger', 3: 'Centre', 4: 'Centre', 5: 'Winger', 6: 'Five-Eight', 7: 'Halfback',
         8: 'Prop', 9: 'Hooker', 10: 'Prop', 11: '2nd Row', 12: '2nd Row', 13: 'Lock'}




In [167]:
# Get all the match data 

team_1_attack = filtered_df[filtered_df[f"{team_1_name} Attack"] != -1][f"{team_1_name} Attack"].mean()
team_1_defense = filtered_df[filtered_df[f"{team_1_name} Defense"] != -1][f"{team_1_name} Defense"].mean()

team_2_attack = filtered_df[filtered_df[f"{team_2_name} Attack"] != -1][f"{team_2_name} Attack"].mean()
team_2_defense = filtered_df[filtered_df[f"{team_2_name} Defense"] != -1][f"{team_2_name} Defense"].mean()


print(f"{team_1_name} Attack: {team_1_attack} Defense: {team_1_defense}\n{team_2_name} Attack: {team_2_attack} Defense: {team_2_defense}")

Eels Attack: 19.625 Defense: 26.5
Sea Eagles Attack: 26.625 Defense: 21.75


In [168]:
def find_player_data(p):
    tries = (p[p['Tries']!= "-"])['Tries'].count()

    tackle_efficiency = p['Tackle Efficiency']
    tackle_efficiency = tackle_efficiency.replace('-', '1.0')
    tackle_efficiency= tackle_efficiency.str.rstrip('%').astype(float) / 100

    
    games = p['Tries'].count() 
    
    try_per_games = tries/games 
    
    return tackle_efficiency.mean(), try_per_games

In [169]:
# Find the top try scorers for each team 

team_1_tef = {}
team_1_tpg = {}

team_2_tef = {}
team_2_tpg = {}

for player_1, player_2, position in zip(team_1, team_2, names.keys()):
    try:
        
        try:
            p_1 = players[player_1]
            tackle_efficiency, try_per_game = find_player_data(p_1)
        except:
             tackle_efficiency, try_per_game = -1, -1
            

        team_1_tef[position] = tackle_efficiency
        team_1_tpg[position] = try_per_game
        
        
        try:
            p_2 = players[player_2]
            tackle_efficiency_2, try_per_game_2 = find_player_data(p_2)
        except:
             tackle_efficiency_2, try_per_game_2  = -1 -1

        team_2_tef[position] = tackle_efficiency_2
        team_2_tpg[position] = try_per_game_2
        
        
        
        print(f"Pos: {position}\tName: {player_1}\tTEF: {tackle_efficiency:.3f}\tTPG: {try_per_game:.3f}")
        print(f"Pos: {position}\tName: {player_2}\tTEF: {tackle_efficiency_2:.3f}\tTPG: {try_per_game_2:.3f}\n")
        
    except Exception as ex:
        print(f'{player_1} not found {ex}')
        p = None 



Pos: 1	Name: Clinton Gutherson	TEF: 0.649	TPG: 0.125
Pos: 1	Name: Tom Trbojevic	TEF: 0.564	TPG: 0.500

Pos: 2	Name: Morgan Harper	TEF: 0.694	TPG: 0.375
Pos: 2	Name: Raymond Tuaimalo Vaega	TEF: 0.615	TPG: 0.000

Pos: 3	Name: Will Penisini	TEF: 0.824	TPG: 0.250
Pos: 3	Name: Tolutau Koula	TEF: 0.782	TPG: 0.500

Pos: 4	Name: Blaize Talagi	TEF: 0.543	TPG: 0.167
Pos: 4	Name: Reuben Garrick	TEF: 0.683	TPG: 0.429

Pos: 5	Name: Sean Russell	TEF: 0.680	TPG: 0.000
Pos: 5	Name: Jaxson Paulo	TEF: 0.791	TPG: 0.333

Pos: 6	Name: Dylan Brown	TEF: 0.851	TPG: 0.125
Pos: 6	Name: Luke Brooks	TEF: 0.776	TPG: 0.125

Pos: 7	Name: Mitchell Moses	TEF: 0.819	TPG: 0.333
Pos: 7	Name: Daly Cherry-Evans	TEF: 0.881	TPG: 0.375

Pos: 8	Name: Reagan Campbell-Gillard	TEF: 0.890	TPG: 0.000
Pos: 8	Name: Taniela Paseka	TEF: 0.907	TPG: 0.000

Pos: 9	Name: Joey Lussick	TEF: 0.893	TPG: 0.250
Pos: 9	Name: Lachlan Croker	TEF: 0.859	TPG: 0.250

Pos: 10	Name: Junior Paulo	TEF: 0.923	TPG: 0.000
Pos: 10	Name: Josh Aloiai	TEF: 0.887

In [170]:
team_2_tpg

{1: 0.5,
 2: 0.0,
 3: 0.5,
 4: 0.42857142857142855,
 5: 0.3333333333333333,
 6: 0.125,
 7: 0.375,
 8: 0.0,
 9: 0.25,
 10: 0.0,
 11: 0.375,
 12: 0.3333333333333333}

In [171]:
avg_tef_1 = np.array(list(team_1_tef.values())).mean() 
avg_tpg_1 = np.array(list(team_1_tpg.values()))
avg_tpg_1 = np.median(avg_tpg_1)

avg_tef_2 = np.array(list(team_2_tef.values())).mean() 
avg_tpg_2 = np.array(list(team_2_tpg.values()))
avg_tpg_2 = np.median(avg_tpg_2)


print(f"{team_1_name} TEF: {avg_tef_1:.2f} TPG: {avg_tpg_1:.2f}")
print(f"{team_2_name} TEF: {avg_tef_2:.2f} TPG: {avg_tpg_2:.2f}")

Eels TEF: 0.81 TPG: 0.17
Sea Eagles TEF: 0.78 TPG: 0.33


In [172]:
# Compare Wingers

winger_left = 2
centre_left = 3

winger_right = 5
centre_right = 4


winger_left_1_tef = team_1_tef[winger_left]
winger_left_2_tef = team_2_tef[winger_left]

centre_left_1_tef = team_1_tef[centre_left]
centre_left_2_tef = team_2_tef[centre_left]


winger_right_1_tef = team_1_tef[winger_right]
winger_right_2_tef = team_2_tef[winger_right]

centre_right_1_tef = team_1_tef[centre_right]
centre_right_2_tef = team_2_tef[centre_right]


if winger_left_1_tef != -1 and centre_left_1_tef != -1:
    # Calculate the average
    left_tef_1 = (winger_left_1_tef + centre_left_1_tef) / 2
else:
    # Take the non-negative value
    left_tef_1 = winger_left_1_tef if winger_left_1_tef != -1 else centre_left_1_tef


if winger_right_1_tef != -1 and centre_right_1_tef != -1:
    # Calculate the average
    right_tef_1 = (winger_right_1_tef + centre_right_1_tef) / 2
else:
    # Take the non-negative value
    right_tef_1 = winger_right_1_tef if winger_right_1_tef != -1 else centre_right_1_tef


if winger_left_2_tef != -1 and centre_left_2_tef != -1:
    # Calculate the average
    left_tef_2 = (winger_left_2_tef + centre_left_2_tef) / 2
else:
    # Take the non-negative value
    left_tef_2 = winger_left_2_tef if winger_left_2_tef != -1 else centre_left_2_tef


if winger_right_2_tef != -1 and centre_right_2_tef != -1:
    # Calculate the average
    right_tef_2 = (winger_right_2_tef + centre_right_2_tef) / 2
else:
    # Take the non-negative value
    right_tef_2 = winger_right_2_tef if winger_right_2_tef != -1 else centre_right_2_tef






# print(f"{winger_left_1_tef}\t{winger_left_2_tef}\t{winger_right_1_tef}\t{winger_right_2_tef}")

# Teams Attack * OP Teams Defense * (teams Avg TPG) * (op teams avg def) * TPG * [OP TEF Winger + Center Average] 
home_bonus = 1.05



win_def_1 = (team_1_attack + team_2_defense)/2

win_def_2 = (team_2_attack + team_1_defense)/2

# Normalise against each other
max_win_def = win_def_1 if win_def_1 > win_def_2 else win_def_2
win_def_1 = win_def_1/max_win_def
win_def_2 = win_def_2/max_win_def


w1 = 0.2
w2 = 0.075
w3 = 0.4
w4 = 0.25
w5 = 0.075

w1_l = (win_def_1*w1 + (avg_tpg_1)*w2 + (team_1_tpg[winger_left])*w3  + (1-left_tef_2)*w4 + (1-avg_tef_2)*w5)*home_bonus
w2_l = win_def_2*w1 + (avg_tpg_2)*w2 + (team_2_tpg[winger_left])*w3  + (1-left_tef_1)*w4+ (1-avg_tef_1)*w5

w1_r = (win_def_1*w1 + (avg_tpg_1)*w2 + (team_1_tpg[winger_right])*w3  + (1-right_tef_2)*w4 + (1-avg_tef_2)*w5)*home_bonus
w2_r = win_def_2*w1 + (avg_tpg_2)*w2 + (team_2_tpg[winger_right])*w3  + (1-right_tef_1)*w4+ (1-avg_tef_1)*w5


w1 = 0.2
w2 = 0.1
w3 = 0.5
w4 = 0.1

w1_fb = (win_def_1*w1 + (avg_tpg_1)*w2 + (team_1_tpg[1])*w3 + (1-avg_tef_2)*w4)*home_bonus
w2_fb = win_def_2*w1 + (avg_tpg_2)*w2 + (team_2_tpg[1])*w3  + (1-avg_tef_1)*w4



# Define the width for each field
width = 10

# Print the header
print(f"{'Team Name':<{width}} {'Attack':>{width}} {'Defense':>{width}} {'TPG':>{width}} {'TEF':>{width}} {'LTEF':>{width}} {'RTEF':>{width}} {'LW APG':>{width}} {'RW APG':>{width}} {'FB APG':>{width}}")

# Print the formatted strings with specified widths for the results


print(f"{team_1_name:<{width}} {team_1_attack:>{width}.2f} {team_1_defense:>{width}.2f} {avg_tpg_1:>{width}.2f} {avg_tef_1:>{width}.2f} {left_tef_1:>{width}.2f} {right_tef_1:>{width}.2f} {team_1_tpg[winger_left]:>{width}.2f} {team_1_tpg[winger_right]:>{width}.2f} {team_1_tpg[1]:>{width}.2f}")
print(f"{team_2_name:<{width}} {team_2_attack:>{width}.2f} {team_2_defense:>{width}.2f} {avg_tpg_2:>{width}.2f} {avg_tef_2:>{width}.2f} {left_tef_2:>{width}.2f} {right_tef_2:>{width}.2f} {team_2_tpg[winger_left]:>{width}.2f} {team_2_tpg[winger_right]:>{width}.2f} {team_2_tpg[1]:>{width}.2f}")
print('\n')

print(f"{team_1_name:<{width}} {team_1[winger_left-1]}: {w1_l:.3f}\n{team_2_name:<{width}} {team_2[winger_left-1]}: {w2_l:.3f}")
print(f"{team_1_name:<{width}} {team_1[winger_right-1]}: {w1_r:.3f}\n{team_2_name:<{width}} {team_2[winger_right-1]}: {w2_r:.3f}")

print('\n')
print(f"{team_1_name:<{width}} {team_1[1-1]}: {w1_fb:.3f}\n{team_2_name:<{width}} {team_2[1-1]}: {w2_fb:.3f}")


Team Name      Attack    Defense        TPG        TEF       LTEF       RTEF     LW APG     RW APG     FB APG
Eels            19.62      26.50       0.17       0.81       0.76       0.61       0.38       0.00       0.12
Sea Eagles      26.62      21.75       0.33       0.78       0.70       0.74       0.00       0.33       0.50


Eels       Morgan Harper: 0.430
Sea Eagles Raymond Tuaimalo Vaega: 0.300
Eels       Sean Russell: 0.263
Sea Eagles Jaxson Paulo: 0.470


Eels       Clinton Gutherson: 0.270
Sea Eagles Tom Trbojevic: 0.502


In [173]:
print(f"Home: {win_def_1:.2f} Away: {win_def_2:.2f}")

Home: 0.78 Away: 1.00
